In [4]:
import json
from collections import defaultdict

def get_counts(sequence):
    counts = {}
    for x in sequence:
        if x in counts:
            counts[x] += 1
        else:
            counts[x] = 1
    return counts

'''使用库函数进行的统计'''
def get_counts2(sequence):
    counts = defaultdict(int)
    for x in sequence:
        counts[x] += 1
    return counts

'''获取前十的访问来源'''
def top_counts(count_dict,n=10):
    value_key_paris = [(count,tz) for tz,count in count_dict.items()]
    value_key_paris.sort()
    return value_key_paris[-n:]

path = 'ch01/usagov_bitly_data2012-03-16-1331923249.txt'

'''使用json的库方法对文件内容进行读取，并存放到数组中'''
records = [json.loads(line)
           for line in open(path)]
records[0]

records[0]['tz']

#print(records[0]['tz'])

time_zone = [rec['tz'] for rec in records
             if 'tz' in rec]

time_zone[:10]

counts = get_counts(time_zone)

counts['America/New_York']

len(time_zone)

top_counts(counts)



[(33, 'America/Sao_Paulo'),
 (35, 'Europe/Madrid'),
 (36, 'Pacific/Honolulu'),
 (37, 'Asia/Tokyo'),
 (74, 'Europe/London'),
 (191, 'America/Denver'),
 (382, 'America/Los_Angeles'),
 (400, 'America/Chicago'),
 (521, ''),
 (1251, 'America/New_York')]

In [6]:
from collections import Counter
'''使用库函数统计前十的区域'''
counts = Counter(time_zone)

counts.most_common(10)

[('America/New_York', 1251),
 ('', 521),
 ('America/Chicago', 400),
 ('America/Los_Angeles', 382),
 ('America/Denver', 191),
 ('Europe/London', 74),
 ('Asia/Tokyo', 37),
 ('Pacific/Honolulu', 36),
 ('Europe/Madrid', 35),
 ('America/Sao_Paulo', 33)]

In [7]:
from pandas import DataFrame,Series
import pandas as pd;import numpy as np

'''使用pandas对时区进行计数'''

frame = DataFrame(records)

'''这里frame的输出形式是摘要视图(summary view)，主要用于较大的
DataFrame对象。frame['tz']所反悔的Series对象有个value_counts方法'''
frame['tz'][:10]

tz_counts = frame['tz'].value_counts()

tz_counts[:10]

America/New_York       1251
                        521
America/Chicago         400
America/Los_Angeles     382
America/Denver          191
Europe/London            74
Asia/Tokyo               37
Pacific/Honolulu         36
Europe/Madrid            35
America/Sao_Paulo        33
Name: tz, dtype: int64

In [8]:
clean_tz = frame['tz'].fillna('Missing')

clean_tz[clean_tz == ''] = 'Unknown'

tz_counts = clean_tz.value_counts()

tz_counts[:10]

In [26]:
from matplotlib.pyplot import *

'''利用tz_counts对象的plot方法即可得到一张水平条形图'''
tz_counts[:10].plot(kind='barh',rot=0)
show()


'Mozilla/5.0 (Windows NT 5.1; rv:10.0.2) Gecko/20100101 Firefox/10.0.2'

In [11]:
frame['a'][1]
'''此外，我们还可以对这种数据进行很多处理。比如：a字段含有执行URL短缩
操作的浏览器、设备、应用程序等信息'''
frame['a'][50]
frame['a'][51]

In [ ]:
results = Series([x.split()[0] for x in frame.a.dropna()])
results[:5]
results.value_counts()[:8]

0               Mozilla/5.0
1    GoogleMaps/RochesterNY
2               Mozilla/4.0
3               Mozilla/5.0
4               Mozilla/5.0
dtype: object